In [2]:
import pandas as pd
import numpy as np
import re
import time
from tqdm import tqdm
from util import pre_process
import pickle
tqdm.pandas()
%matplotlib inline
from nltk import word_tokenize,pos_tag

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

path='../data/'

# 加载数据（获取关键句和预处理）

In [2]:
train=pd.read_csv(path+'train_release.csv')
test=pd.read_csv(path+'validation_release.csv'
canditate=pd.read_csv(path+'candidate_paper.csv'))
train=train[~train['description_id'].isnull()]

In [3]:
'获取关键句函数'
def digest(text):
    backup = text[:]
    text = text.replace('al.', '').split('. ')
    t=''
    pre_text=[]
    len_text=len(text)-1
    add=True
    pre=''
    while len_text>=0:
        index=text[len_text]
        index+=pre
        if len(index.split(' '))<=3 :
            add=False
            pre=index+pre
        else:
            add=True
            pre=''
        if add:
            pre_text.append(index)
        len_text-=1
    if len(pre_text)==0:
        pre_text=text
    pre_text.reverse()
    for index in pre_text:
        if index.find('[**##**]') != -1:
            index = re.sub(r'[\[|,]+\*\*\#\#\*\*[\]|,]+','',index)
            index+='. '
            t+=index
    return t
train['key_text']=train['description_text'].apply(lambda x:digest(x))
test['key_text']=test['description_text'].apply(lambda x:digest(x))

(43956, 3)
(19018, 2)


In [ ]:
train['key_text_pre']=train['key_text'].progress_apply(lambda x:' '.join(pre_process(x)))
test['key_text_pre']=test['key_text'].progress_apply(lambda x:' '.join(pre_process(x)))


In [ ]:
train.to_csv(path+'train_pre.csv',index=False)
test.to_csv(path+'test_pre.csv',index=False)

In [ ]:
canditate['title_pro']=canditate['title'].progress_apply(lambda x:' '.join(pre_process(x)))

In [ ]:
canditate['abstract_pre']=canditate['abstract'].progress_apply(lambda x:' '.join(pre_process(x) if str(x)!='nan' else 'none'))

In [ ]:
canditate.to_csv(path+'candidate_paper_pre.csv',index=False)

# 获得训练集和测试集

In [5]:
train_pre=pd.read_csv(path+'train_pre.csv')
test_pre=pd.read_csv(path+'test_pre.csv')
canditate_pre=pd.read_csv(path+'candidate_paper_pre.csv')

In [ ]:
train_pre['label']=1
train_pre=train_pre.drop_duplicates('description_id')

In [8]:
'获取召回数据函数'
def get_recall(path):
    data=pd.read_csv(path,header=None,names=['description_id']+[i for i in range(1,21)]).drop_duplicates('description_id').values
    description_id = []
    pid = []
    for i in data:
        description_id.extend([i[0]] * 20)
        pid.extend(list(i[1:]))
    return  pd.DataFrame({'description_id':description_id,'pid':pid})

In [10]:
'召回pid'
re_path='data/test_pairs_20number_tfidf.csv'
test_recall=get_recall(re_path)
test_data=test_pre.merge(test_recall,on='description_id',how='inner')
re_path='data/train_pairs_20number_tfidf.csv'
train_recall=get_recall(re_path)
train_data=train_pre.merge(train_recall,on='description_id',how='inner')
train_data['label']=list(map(lambda x,y: int(x==y),train_data['pid'],train_data['paper_id']))

In [11]:
train_data.shape

(879100, 8)

In [12]:
test_data.shape

(380360, 6)

In [13]:
'删去没用被召回的数据'
temp=train_data[['description_id','label']].groupby('description_id')['label'].apply(lambda x:len(set(list(x)))).reset_index()
temp.columns=['description_id','label']
ids=temp[temp['label']==2]['description_id'].values
train_data=train_data[train_data['description_id'].isin(ids)]

In [14]:
'召回获取没用被召回的论文'
bad_ids=temp[temp['label']==1]['description_id'].values
train_data_bad=train_pre[train_pre['description_id'].isin(bad_ids)]
train_data_bad['pid']=train_data_bad['paper_id'].values

In [15]:
print(train_data_bad.shape)
print(train_data.shape)

(17855, 8)
(522000, 8)


In [16]:
train_data=pd.concat([train_data,train_data_bad],axis=0,sort=True)

In [19]:
'合并对应论文'
train_data['paper_id']=train_data['pid'].values
train_data.pop('pid')
train_data=train_data.merge(canditate_pre,on='paper_id',how='left')
test_data['paper_id']=test_data['pid'].values
test_data.pop('pid')
test_data=test_data.merge(canditate_pre,on='paper_id',how='left')

In [16]:
train_data.to_csv('train_set/train_data_tfidf_20_addbad.csv',index=False)
test_data.to_csv('train_set/test_data_tfidf_20_addbad.csv',index=False)

NameError: name 'train_data' is not defined

# merge set

In [3]:
import pandas as pd

In [11]:
test_data_tf=pd.read_csv('train_set/test_data_tfidf_20.csv')
train_data_tf=pd.read_csv('train_set/train_data_tfidf_20.csv')
print(test_data_tf.shape)
print(train_data_tf.shape)

(522000, 13)


In [13]:
test_data_bm25=pd.read_csv('train_set/test_data_bm25_20.csv')[list(test_data_tf.columns)]
train_data_bm25=pd.read_csv('train_set/train_data_bm25_20.csv')[list(train_data_tf.columns)]
print(test_data_bm25.shape)
print(train_data_bm25.shape)

(542320, 13)


In [14]:
train_data_merge=pd.concat([train_data_bm25,train_data_tf],axis=0,sort=True)
test_data_merge=pd.concat([test_data_bm25,test_data_tf],axis=0,sort=True)
print(test_data_merge.shape)
print(train_data_merge.shape)

(570540, 12)
(838252, 13)


In [15]:
train_data_merge=train_data_merge.drop_duplicates()
test_data_merge=test_data_merge.drop_duplicates()

In [16]:
print(test_data_merge.shape)
print(train_data_merge.shape)

(459428, 12)
(838252, 13)


In [13]:
train_data_merge=train_data_merge[train_data_merge['key_text_pre'].notnull()]

In [15]:
train_data_merge.to_csv('train_set/train_data_merge_bm25_tfidf_20_add.csv',index=False)
test_data_merge.to_csv('train_set/test_data_merge_bm25_tfidf_20.csv',index=False)

In [9]:
train_data_merge.label.value_counts(True)

0    0.965089
1    0.034911
Name: label, dtype: float64